In [1]:
# pip install geopandas

In [2]:
# pip install pyarrow

In [3]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

from google.cloud import bigquery

import geopy
import folium
from uszipcode import SearchEngine
from uszipcode import Zipcode
import numpy as np
import geopandas

import ipywidgets as widgets

In [4]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [5]:
conda install nodejs

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [6]:
# conda install npm

In [7]:
# Changing the settings to override this file path in the environment
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=r"\Users\kenda\OneDrive\Documents\MADS\591MilestoneI\SanFranciscoBikeShare\sf-bike-321222-7e949b330c94.json"

In [8]:
# all distinct station pairs with lat lons
client = bigquery.Client()

query = """
    SELECT DISTINCT start_station_id, start_station_longitude, start_station_latitude,
        end_station_id, end_station_longitude, end_station_latitude
    FROM `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
    WHERE EXTRACT(YEAR from start_date)=2017
"""

query_job = client.query(query)  # Make an API request.

In [9]:
latlong2017 = query_job.to_dataframe()

In [10]:
latlong2017.shape

(20027, 6)

In [11]:
## trips dataframe

query = """
    SELECT *
    FROM `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
    WHERE EXTRACT(YEAR from start_date)=2017
"""

query_job = client.query(query)  # Make an API request.

In [12]:
trips = query_job.to_dataframe()

In [13]:
trips.shape

(519700, 21)

In [14]:
## station dataframe

client = bigquery.Client()

query = """
    SELECT *
    FROM `bigquery-public-data.san_francisco_bikeshare.bikeshare_station_info`
"""

query_job = client.query(query)  # Make an API request.

In [15]:
stations = query_job.to_dataframe()

In [16]:
#define zipcode search function
search = SearchEngine(simple_zipcode=True)

def get_zipcode(lat, lon):
    result = search.by_coordinates(lat = lat, lng = lon, returns = 1)
    return result[0].zipcode

#add new column with generated zip-code
stations['zipcode'] = stations.apply(lambda x: get_zipcode(x.lat,x.lon), axis=1)

In [17]:
def get_tripzips(latlonDF, start_lat, start_lon, end_lat, end_lon):
    #get zipcode for each start station
    latlonDF['start_zipcode'] = latlonDF.apply(lambda x: get_zipcode(x[start_lat],x[start_lon]), axis=1)
    
    #get zipcode for each end station
    latlonDF['end_zipcode'] = latlonDF.apply(lambda x: get_zipcode(x[end_lat],x[end_lon]), axis=1)

    return latlonDF

#### Get zipcodes for unique start and end station pairs.

In [18]:
latlons = get_tripzips(latlong2017, "start_station_latitude", 'start_station_longitude', 'end_station_latitude', 'end_station_longitude')

In [19]:
latlons.head()

,start_station_id,start_station_longitude,start_station_latitude,end_station_id,end_station_longitude,end_station_latitude,start_zipcode,end_zipcode
0,240,-122.258804,37.866043,18,-122.260172,37.850222,94705,94705
1,8,-122.398525,37.799953,8,-122.398525,37.799953,94104,94104
2,318,-121.888979,37.330698,310,-121.885660,37.335885,95113,95113
3,270,-122.289415,37.853489,251,-122.279720,37.870555,94702,94703
4,214,-122.275733,37.823321,158,-122.263490,37.833279,94608,94609


In [20]:
# tag trips with start and end zipcodes
tripsWithZip = trips.merge(latlons, on = ["start_station_id", "end_station_id"])

In [21]:
#drop duplicate columns

tripsWithZip = tripsWithZip.drop(columns = ['start_station_longitude_y',
       'start_station_latitude_y', 'end_station_longitude_y',
       'end_station_latitude_y'])

In [22]:
tripsWithZip.head()

,trip_id,duration_sec,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_number,zip_code,...,start_station_longitude_x,end_station_latitude_x,end_station_longitude_x,member_birth_year,member_gender,bike_share_for_all_trip,start_station_geom,end_station_geom,start_zipcode,end_zipcode
0,14172017123118141200,762,2017-12-31 18:14:12+00:00,Haste St at Telegraph Ave,240,2017-12-31 18:26:54+00:00,Telegraph Ave at Alcatraz Ave,18,1417,None,...,-122.258804,37.850222,-122.260172,1985.0,Female,None,POINT(-122.2588044 37.8660431),POINT(-122.260172367096 37.8502218744968),94705,94705
1,1692017121118313900,505,2017-12-11 18:31:39+00:00,Haste St at Telegraph Ave,240,2017-12-11 18:40:04+00:00,Telegraph Ave at Alcatraz Ave,18,169,None,...,-122.258804,37.850222,-122.260172,1992.0,Male,None,POINT(-122.2588044 37.8660431),POINT(-122.260172367096 37.8502218744968),94705,94705
2,12942017083119530100,541,2017-08-31 19:53:01+00:00,Haste St at Telegraph Ave,240,2017-08-31 20:02:03+00:00,Telegraph Ave at Alcatraz Ave,18,1294,None,...,-122.258804,37.850222,-122.260172,1992.0,Male,None,POINT(-122.2588044 37.8660431),POINT(-122.260172367096 37.8502218744968),94705,94705
3,12292017081210375300,532,2017-08-12 10:37:53+00:00,Haste St at Telegraph Ave,240,2017-08-12 10:46:45+00:00,Telegraph Ave at Alcatraz Ave,18,1229,None,...,-122.258804,37.850222,-122.260172,1987.0,Male,None,POINT(-122.2588044 37.8660431),POINT(-122.260172367096 37.8502218744968),94705,94705
4,12232017090102340300,334,2017-09-01 02:34:03+00:00,Haste St at Telegraph Ave,240,2017-09-01 02:39:38+00:00,Telegraph Ave at Alcatraz Ave,18,1223,None,...,-122.258804,37.850222,-122.260172,1992.0,Male,None,POINT(-122.2588044 37.8660431),POINT(-122.260172367096 37.8502218744968),94705,94705


##### What percentage of 2017 rides start and end in the same zip code?

In [23]:
sameZipCode = tripsWithZip[tripsWithZip["start_zipcode"]==tripsWithZip["end_zipcode"]]

In [24]:
sameZipCode.shape[0]/tripsWithZip.shape[0]

0.266823167211853

**26.7% of 2017 rides start and end in the same zip code

### Folium Exploration

In [142]:
''' Mean ride time in seconds by start zipcode'''

start_duration = tripsWithZip.groupby(["start_zipcode"])["duration_sec"].mean().reset_index().set_index("start_zipcode")


''' read in shapes for SF zipcodes '''
zipcodes = geopandas.read_file("SFZipCodes.json")
zipcodes = zipcodes.set_index("ZIP")

''' Merge zip code geometries back to trip data '''
duration_zips = start_duration.merge(zipcodes, how = "inner", left_index=True, right_index=True)

In [132]:
# ''' Merge zip code geometries back to trip data '''
# duration_zips = start_duration.merge(zipcodes, how = "inner", left_index=True, right_index=True)

In [134]:
# start_duration = start_duration.reset_index()

In [179]:
tripsWithZip.head()

,trip_id,duration_sec,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_number,zip_code,...,end_station_latitude_x,end_station_longitude_x,member_birth_year,member_gender,bike_share_for_all_trip,start_station_geom,end_station_geom,start_zipcode,end_zipcode,duration_min
0,14172017123118141200,762,2017-12-31 18:14:12+00:00,Haste St at Telegraph Ave,240,2017-12-31 18:26:54+00:00,Telegraph Ave at Alcatraz Ave,18,1417,None,...,37.850222,-122.260172,1985.0,Female,None,POINT(-122.2588044 37.8660431),POINT(-122.260172367096 37.8502218744968),94705,94705,12.700000
1,1692017121118313900,505,2017-12-11 18:31:39+00:00,Haste St at Telegraph Ave,240,2017-12-11 18:40:04+00:00,Telegraph Ave at Alcatraz Ave,18,169,None,...,37.850222,-122.260172,1992.0,Male,None,POINT(-122.2588044 37.8660431),POINT(-122.260172367096 37.8502218744968),94705,94705,8.416667
2,12942017083119530100,541,2017-08-31 19:53:01+00:00,Haste St at Telegraph Ave,240,2017-08-31 20:02:03+00:00,Telegraph Ave at Alcatraz Ave,18,1294,None,...,37.850222,-122.260172,1992.0,Male,None,POINT(-122.2588044 37.8660431),POINT(-122.260172367096 37.8502218744968),94705,94705,9.016667
3,12292017081210375300,532,2017-08-12 10:37:53+00:00,Haste St at Telegraph Ave,240,2017-08-12 10:46:45+00:00,Telegraph Ave at Alcatraz Ave,18,1229,None,...,37.850222,-122.260172,1987.0,Male,None,POINT(-122.2588044 37.8660431),POINT(-122.260172367096 37.8502218744968),94705,94705,8.866667
4,12232017090102340300,334,2017-09-01 02:34:03+00:00,Haste St at Telegraph Ave,240,2017-09-01 02:39:38+00:00,Telegraph Ave at Alcatraz Ave,18,1223,None,...,37.850222,-122.260172,1992.0,Male,None,POINT(-122.2588044 37.8660431),POINT(-122.260172367096 37.8502218744968),94705,94705,5.566667


In [180]:
# convert seconds to minutes
tripsWithZip['duration_min'] = tripsWithZip['duration_sec']/60


# get mean ride time and number of rides
rideStatsByZip = tripsWithZip.groupby(['start_zipcode'])['duration_min'].agg([np.mean, np.size]).reset_index().rename(
                                                                                        columns = {'mean': 'avg_mins',
                                                                                                  'size': 'num_rides'})

In [181]:
rideStatsByZip.head()

,start_zipcode,avg_mins,num_rides
0,94102,17.356176,51586.0
1,94103,17.358828,44083.0
2,94104,19.799288,54783.0
3,94105,16.554190,91814.0
4,94108,25.923428,12648.0


In [188]:
''' Initialize SF Map '''

sf_map = folium.Map(location=[37.807180,-122.352798], zoom_start=11) # set location to San Fran captial

# Put marker on map on San Francisco 'technical' lat, long
folium.Marker(
    location=[37.807180,-122.352798],
    popup='Welcome to <b>San Francisco</b>', # Pop up text once clicked
    tooltip = "Click for more" # Mouse-over pop-up
).add_to(sf_map) # Add to map object.  You can continue to add content once you make a map object ('sf_map')



choropleth = folium.Choropleth(geo_data="SFZipCodes.json", # Input San Fran ZIP codes from geojson
             data=rideStatsByZip, # trips mapped to zipcode
             columns=['start_zipcode', 'avg_mins'], # Zip code is here for matching the geojson zipcode, duration is length of ride
             key_on='feature.properties.ZIP', # This path contains zipcodes in str type, this should match with our start_zipcode column
             fill_color='BuPu', fill_opacity=0.9, line_opacity=0.2,
             legend_name='Average Ride Duration (Minutes)', 
            nan_fill_color='grey',
             nan_fill_opacity=0.1).add_to(sf_map)

title_html = '''
                 <h3 align="center" style="font-size:16px"><b>Average Ride Time (minutes) by Ride Start Zip Code</b></h3>
                 '''  

sf_map.get_root().html.add_child(folium.Element(title_html));

In [189]:
sf_map

### Insights

Most rides are pretty short, especially towards the epicenter of SF and Oakland. However, the there are some areas where people tend to take longer rides, further out from downtown.

### Ride Route Popularity

In [195]:
int_slider = widgets.IntSlider(
    value = 500,
    min=0,
    max=10000,
    step=1,
    description='Min Number of Rides per Route'
#     disabled=False,
#     continuous_update=False,
#     orientation='horizontal',
#     readout=True,
#     readout_format='d'
)

int_slider

IntSlider(value=500, description='Min Number of Rides per Route', max=10000)

In [213]:
m = folium.Map(location=[37.807180,-122.352798], zoom_start=12)


trips['start_station'] = trips[['start_station_latitude', 'start_station_longitude']].apply(tuple, axis=1)

trips['end_station'] = trips[['end_station_latitude', 'end_station_longitude']].apply(tuple, axis=1)


station_freq = trips.groupby(['start_station', 'end_station'])['trip_id'].count().reset_index().rename(columns = {'trip_id': "trip_count"})


station_freq_toproutes = station_freq[station_freq['trip_count']>=int_slider.value]

station_freq_toproutes.apply(lambda row:folium.PolyLine([row["start_station"], 
                                                  row["end_station"]], 
                                                  weight = (row['trip_count']/trips.shape[0])*800,
                                                 opacity = 0.9).add_to(m),
                                                                           axis = 1);


title_html = f'''
                 <h3 align="center" style="font-size:16px"><b>Route Popularity </b></h3>
                <h4 align="center" style="font-size:12px"><b>Route Minimum: {int_slider.value} rides</b></h4>

                 '''

m.get_root().html.add_child(folium.Element(title_html));

In [214]:
m

#### Insights

Popular routes tend to be along/near the water. There are several stations that seem to be "hubs". Biking is particularly popular in the Rincon Hill area and the financial district in Oakland.